**МОНИТОРИНГ СОЦИАЛЬНЫХ ВЫПЛАТ**

In [1]:
import pandas as pd
from datetime import date
import os
import sys
from IPython.display import clear_output
from geopy.distance import geodesic as GD
import shutil # упаковка в архив
import warnings # игнорирование предупреждения о формате при импорте из excel

warnings.simplefilter("ignore")

# функции-хуюнкции
def lat(row):
    if row["r_latitude"] == "no_data" or row["r_longtitude"] == "no_data" or row["r_region_name"] == "no_data" or row["r_area"] == "no_data":
        if row["d_latitude"] == "no_data" or row["d_longtitude"] == "no_data" or row["d_region_name"] == "no_data" or row["d_area"] == "no_data":
            if row["mo_latitude"] == "no_data" or row["mo_longtitude"] == "no_data" or row["mo_region_name"] == "no_data" or row["mo_area"] == "no_data":
                return "no_data"
            else:
                return row["mo_latitude"]
        else:
            return row["d_latitude"]
    else:
        return row["r_latitude"]
    
def lon(row):
    if row["r_latitude"] == "no_data" or row["r_longtitude"] == "no_data" or row["r_region_name"] == "no_data" or row["r_area"] == "no_data":
        if row["d_latitude"] == "no_data" or row["d_longtitude"] == "no_data" or row["d_region_name"] == "no_data" or row["d_area"] == "no_data":
            if row["mo_latitude"] == "no_data" or row["mo_longtitude"] == "no_data" or row["mo_region_name"] == "no_data" or row["mo_area"] == "no_data":
                return "no_data"
            else:
                return row["mo_longtitude"]
        else:
            return row["d_longtitude"]
    else:
        return row["r_longtitude"]

def pref(row):
    if row["r_latitude"] == "no_data" or row["r_longtitude"] == "no_data" or row["r_region_name"] == "no_data" or row["r_area"] == "no_data":
        if row["d_latitude"] == "no_data" or row["d_longtitude"] == "no_data" or row["d_region_name"] == "no_data" or row["d_area"] == "no_data":
            if row["mo_latitude"] == "no_data" or row["mo_longtitude"] == "no_data" or row["mo_region_name"] == "no_data" or row["mo_area"] == "no_data":
                return "no_data"
            else:
                return row["mo_prefix"].lower()
        else:
            return row["d_prefix"].lower()
    else:
        return row["r_prefix"].lower()

def regn(row):
    if row["r_latitude"] == "no_data" or row["r_longtitude"] == "no_data" or row["r_region_name"] == "no_data" or row["r_area"] == "no_data":
        if row["d_latitude"] == "no_data" or row["d_longtitude"] == "no_data" or row["d_region_name"] == "no_data" or row["d_area"] == "no_data":
            if row["mo_latitude"] == "no_data" or row["mo_longtitude"] == "no_data" or row["mo_region_name"] == "no_data" or row["mo_area"] == "no_data":
                return "no_data"
            else:
                return row["mo_region_name"].lower()
        else:
            return row["d_region_name"].lower()
    else:
        return row["r_region_name"].lower()

def ara(row):
    if row["r_latitude"] == "no_data" or row["r_longtitude"] == "no_data" or row["r_region_name"] == "no_data" or row["r_area"] == "no_data":
        if row["d_latitude"] == "no_data" or row["d_longtitude"] == "no_data" or row["d_region_name"] == "no_data" or row["d_area"] == "no_data":
            if row["mo_latitude"] == "no_data" or row["mo_longtitude"] == "no_data" or row["mo_region_name"] == "no_data" or row["mo_area"] == "no_data":
                return "no_data"
            else:
                return row["mo_area"].lower()
        else:
            return row["d_area"].lower()
    else:
        return row["r_area"].lower()

def cut (coord):
    coord_0 = str(coord).replace(".", "")
    while len(coord_0) < 5:
        coord_0 += "0"
    return coord_0[:3]

In [2]:
# пути к файлам и версии справочников (из названия файлов)
dir_path = os.getcwd()
dir_ = os.listdir(dir_path.replace("\\", "/"))
for file in dir_:
    if file.split("_")[0].split(".")[-1] == "1461":
        path_mo = dir_path.replace("\\", "/") + "/" + file
        if file.split("_")[-1].split(".")[-1] == "xlsx":
            mo_version = (
                str(file.split("_")[-1].split(".")[0])
                + "."
                + str(file.split("_")[-1].split(".")[1])
            )
    if file.split("_")[0].split(".")[-1] == "114":
        path_dept = dir_path.replace("\\", "/") + "/" + file
        if file.split("_")[-1].split(".")[-1] == "xlsx":
            sp_version = (
                str(file.split("_")[-1].split(".")[0])
                + "."
                + str(file.split("_")[-1].split(".")[1])
            )
    if file.split("_")[0].split(".")[-1] == "115":
        path_room = dir_path.replace("\\", "/") + "/" + file
        if file.split("_")[-1].split(".")[-1] == "xlsx":
            room_version = (
                str(file.split("_")[-1].split(".")[0])
                + "."
                + str(file.split("_")[-1].split(".")[1])
            )
    if file.split("_")[0].split(".")[-1] == "1123":
        path_address = dir_path.replace("\\", "/") + "/" + file
        if file.split("_")[-1].split(".")[-1] == "xlsx":
            address_version = (
                str(file.split("_")[-1].split(".")[0])
                + "."
                + str(file.split("_")[-1].split(".")[1])
            )
# path_save_sp = dir_path.replace("\\", "/") + "/sp.csv"
# path_save_room = dir_path.replace("\\", "/") + "/room.csv"
print("Версии справочников, извлечённые из названий файлов:")
print(
    f"\t- справочник МО: {mo_version}\n\t- справочник СП: {sp_version}\n\t- справочник кабинетов: {room_version}\n\t- справочник адресов: {address_version}"
)
if path_mo.split('.')[-1] == 'zip' or path_dept.split('.')[-2] == 'zip' or path_room.split('.')[-2] == 'zip':
    print('Скачанные архивы не распакованы, программа завершает работу.\nПопробуйте распаковать архивы.')
    sys.exit()

print('Загрузка справочников:')

df_mo = pd.read_excel(
    path_mo, sheet_name="Справочник", index_col=None, dtype="str"
)
print("\t- загружен справочник МО")
df_dept = pd.read_excel(
    path_dept, sheet_name="Справочник", index_col=None, dtype="str"
)
print("\t- загружен справочник СП")
df_room = pd.read_excel(
    path_room, sheet_name="Справочник", index_col=None, dtype="str"
)
print("\t- загружен справочник кабинетов")
df_address = pd.read_excel(
    path_address, sheet_name="Справочник", index_col=None, dtype="str"
)
print("\t- загружен справочник адресов")

Версии справочников, извлечённые из названий файлов:
	- справочник МО: 6.1258
	- справочник СП: 2.1616
	- справочник кабинетов: 2.1551
	- справочник адресов: 1.7
Загрузка справочников:
	- загружен справочник МО
	- загружен справочник СП
	- загружен справочник кабинетов
	- загружен справочник адресов


In [3]:
# локальные справочники
df_reg_dict = pd.read_csv("C:/work/scripts/справочники/social_benefits_reg.csv", sep=";")
df_reg_dict["reg_nsi"] = df_reg_dict["reg_nsi"].str.lower().str.replace(r'\W', '', regex=True)
df_reg_dict["reg_rosstat"] = df_reg_dict["reg_rosstat"].str.lower().str.replace(r'\W', '', regex=True)
df_clusters = pd.read_csv("C:/work/scripts/справочники/c&t.csv", sep=";")
df_clusters["region_name"] = df_clusters["region_name"].str.lower().str.replace(r'\W', '', regex=True)
df_clusters["pref"] = df_clusters["pref"].str.lower().str.replace(r'\W', '', regex=True)
df_clusters["area"] = df_clusters["area"].str.lower().str.replace(r'\W', '', regex=True)

In [4]:
# преобразование
df_room_s = (
    df_room[
        [
            "mo_oid",
            "depart_oid",
            "oid",
            "building_address_latitude",
            "building_address_longtitude",
            "building_address_region_name",
            "building_address_prefix_area",
            "building_address_area_name",
        ]
    ]
    .drop(index=0)
    .rename(columns={
        "oid": "room_oid",
        "building_address_latitude": "r_latitude",
        "building_address_longtitude": "r_longtitude",
        "building_address_region_name": "r_region_name",
        "building_address_prefix_area": "r_prefix",
        "building_address_area_name": "r_area",
        })
)
df_dept_s = (
    df_dept[
        [
            "depart_oid",
            "building_address_latitude",
            "building_address_longtitude",
            "building_address_region_name",
            "building_address_prefix_area",
            "building_address_area_name",
        ]
    ]
    .drop(index=0)
    .rename(
        columns={
            "building_address_latitude": "d_latitude",
            "building_address_longtitude": "d_longtitude",
            "building_address_region_name": "d_region_name",
            "building_address_prefix_area": "d_prefix",
            "building_address_area_name": "d_area",
        }
    )
)
df_mo_s = (
    df_mo[
        [
            "oid",
            "regionName",
            "latitude",
            "longtitude",
            "addrRegionName",
            "prefixArea",
            "areaName",
        ]
    ]
    .drop(index=0)
    .rename(columns={
        "oid": "mo_oid",
        "regionName": "mo_reg_name",
        "latitude": "mo_latitude",
        "longtitude": "mo_longtitude",
        "addrRegionName": "mo_region_name",
        "areaName": "mo_area",
        "prefixArea": "mo_prefix",
        })
)

In [12]:
%%time

# формирование адекватного адреса
df_adr = df_room_s.merge(df_mo_s, on="mo_oid", how="outer").merge(df_dept_s, on="depart_oid", how="outer")
df_adr = df_adr.fillna("no_data")
df_adr["lat"] = df_adr.apply(lat, axis=1)
df_adr["lon"] = df_adr.apply(lon, axis=1)
df_adr["reg"] = df_adr.apply(regn, axis=1).str.replace('город ', '').str.replace(r'\W', '', regex=True).str.replace('ё', 'е')
df_adr["pref"] = df_adr.apply(pref, axis=1).str.replace(r'\W', '', regex=True).str.replace('ё', 'е')
df_adr["area"] = df_adr.apply(ara, axis=1).str.replace('город ', '').str.replace(r'\W', '', regex=True).str.replace('ё', 'е')

df_adr_s = df_adr[["mo_oid", "depart_oid", "room_oid", "lat", "lon", "reg", "pref", "area"]]
df_adr_s = df_adr_s.merge(df_reg_dict, left_on="reg", right_on="reg_nsi", how="left")
df_adr_s.drop("reg_nsi", axis=1, inplace=True)
# объединение
# df_pclust = df_adr_s.merge(df_clusters, left_on=["reg_rosstat", "area"], right_on=["region_name", "area"], how="left")


CPU times: total: 16.3 s
Wall time: 1min 9s


In [7]:
# План "А", отвергнутый руководством
# df_pclust["pop_cluster"] = df_pclust["pop_cluster"].fillna(3)
# df_pclust["population"] = df_pclust["population"].fillna(0)
# df_pclust["pop_cluster"] = df_pclust["pop_cluster"].astype("int")
# df_pclust["population"] = df_pclust["population"].astype("int")
# df_pclust.loc[(df_pclust["reg"] == "москва")|(df_pclust["reg"] == "санктпетербург")|(df_pclust["reg"] == "севастополь"), "pop_cluster"] = 1
# df_pclust

**АНТОНОВ ОГОНЬ**   
(План "Б")

In [6]:
df_big_pop = pd.read_csv("C:/work/scripts/справочники/big_population.csv", sep=";", index_col=None)
df_big_pop["region"] = df_big_pop["region"].str.lower().str.replace(r'\W', '', regex=True).str.replace('ё', 'е')
df_big_pop["municipality"] = df_big_pop["municipality"].str.lower().str.replace(r'\W', '', regex=True).str.replace('ё', 'е')
df_big_pop["settlement"] = df_big_pop["settlement"].str.lower().str.replace(r'\W', '', regex=True).str.replace('ё', 'е')
df_big_pop["type"] = df_big_pop["type"].str.lower().str.replace(r'\W', '', regex=True).str.replace('ё', 'е')
df_big_pop = df_big_pop.drop("id", axis=1)

In [13]:
df_adr_s["lat"] = df_adr_s["lat"].str.replace("no_data", "0")
df_adr_s["lon"] = df_adr_s["lon"].str.replace("no_data", "0")
df_adr_s["lat"] = df_adr_s["lat"].astype("float")
df_adr_s["lon"] = df_adr_s["lon"].astype("float")
df_adr_s["region_found"] = ""
df_adr_s["municipality_found"] = ""
df_adr_s["settlement_found"] = ""
df_adr_s["settelment_type_found"] = ""
df_adr_s["population_found"] = ""
df_adr_s["children_found"] = ""
df_adr_s["D_km_found"] = ""


In [14]:
df_adr_s

,mo_oid,depart_oid,room_oid,lat,lon,reg,pref,area,reg_rosstat,region_found,municipality_found,settlement_found,settelment_type_found,population_found,children_found,D_km_found
0,1.2.643.5.1.13.13.12.2.42.3982,1.2.643.5.1.13.13.12.2.42.3982.0.87331,1.2.643.5.1.13.13.12.2.42.3982.0.87331.26572,53.751255,87.136178,кемеровскаяобласть,г,новокузнецк,кемеровскаяобласть,,,,,,,
1,1.2.643.5.1.13.13.12.2.42.3982,1.2.643.5.1.13.13.12.2.42.3982.0.87331,1.2.643.5.1.13.13.12.2.42.3982.0.87331.26573,53.751255,87.136178,кемеровскаяобласть,г,новокузнецк,кемеровскаяобласть,,,,,,,
2,1.2.643.5.1.13.13.12.2.42.3982,1.2.643.5.1.13.13.12.2.42.3982.0.87331,1.2.643.5.1.13.13.12.2.42.3982.0.87331.26421,53.751255,87.136178,кемеровскаяобласть,г,новокузнецк,кемеровскаяобласть,,,,,,,
3,1.2.643.5.1.13.13.12.2.42.3982,1.2.643.5.1.13.13.12.2.42.3982.0.87331,1.2.643.5.1.13.13.12.2.42.3982.0.87331.33475,53.751255,87.136178,кемеровскаяобласть,г,новокузнецк,кемеровскаяобласть,,,,,,,
4,1.2.643.5.1.13.13.12.2.42.3982,1.2.643.5.1.13.13.12.2.42.3982.0.87331,1.2.643.5.1.13.13.12.2.42.3982.0.87331.33476,53.751255,87.136178,кемеровскаяобласть,г,новокузнецк,кемеровскаяобласть,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
963777,no_data,1.2.643.5.1.13.13.12.2.52.5060.0.454246,no_data,57.129626,45.145670,нижегородскаяобласть,рп,красныебаки,нижегородскаяобласть,,,,,,,
963778,no_data,1.2.643.5.1.13.13.12.2.77.12482.0.461797,no_data,0.000000,0.000000,no_data,no_data,no_data,no_data,,,,,,,
963779,no_data,1.2.643.5.1.13.13.12.2.77.1270.0.471279,no_data,0.000000,0.000000,no_data,no_data,no_data,no_data,,,,,,,
963780,no_data,1.2.643.5.1.13.13.12.2.71.15515.0.450889,no_data,53.978838,38.530360,тульскаяобласть,г,кимовск,тульскаяобласть,,,,,,,


In [99]:
%%time

# аццкий код, поиск только с учётом названия населённого пункта
n = 0
lim = 500
for index, row in df_adr_s.iterrows():
    try:
        if index < lim:
            D_km = 100500
            region = "no_data"
            municipality = "no_data"
            settlement = "no_data"
            settelment_type = "no_data"
            population = "no_data"
            children = "no_data"
            pos_0 = (row["lat"], row["lon"])
            df_tmp = df_big_pop[df_big_pop["region"] == row["reg_rosstat"]]
            for i, r in df_tmp.iterrows():
                pos_tmp = (r["latitude_dd"], r["longitude_dd"])
                if row["area"] == r["settlement"]:
                    D_tmp = GD(pos_0, pos_tmp).km
                    if D_tmp < D_km:
                        region = r["region"]
                        municipality = r["municipality"]
                        settlement = r["settlement"]
                        settelment_type = r["type"]
                        population = r["population"]
                        children = r["children"]
                        D_km = D_tmp
            df_adr_s.at[index, "region_found"] = region
            df_adr_s.at[index, "municipality_found"] = municipality
            df_adr_s.at[index, "settlement_found"] = settlement
            df_adr_s.at[index, "settelment_type_found"] = settelment_type
            df_adr_s.at[index, "population_found"] = population
            df_adr_s.at[index, "children_found"] = children
            df_adr_s.at[index, "D_km_found"] = round(D_km, 3)
            n += 1
            clear_output()
            print(f"обработана строка {n} из {df_adr_s.shape[0]} при лимите {lim}\t\t{round(n*100/df_adr_s.shape[0], 3)} %", flush=True)
        else:
            break
    except Exception as e:
        print(f"строка {n} ошибка {e}")
print("сохранение")
df_adr_s.to_csv("C:/Users/Admin/Desktop/RESULT.csv", sep=";", encoding="utf-8-sig", index=False)
print("результат сохранён")

обработана строка 500 из 964951 при лимите 500		0.052 %
результат сохранён
CPU times: total: 8.22 s
Wall time: 37.7 s


In [71]:
%%time

# ещё более аццкий код-2, поиск с учётом названия населённого пункта, а если такого нет - только по координатам
n = 0
lim = 500
for index, row in df_adr_s.iterrows():
    try:
        if index < lim:
            D_km = 100500
            region = "no_data"
            municipality = "no_data"
            settlement = "no_data"
            settelment_type = "no_data"
            population = "no_data"
            children = "no_data"
            pos_0 = (row["lat"], row["lon"])
            df_tmp = df_big_pop[df_big_pop["region"] == row["reg_rosstat"]]
            if row["area"] in (df_tmp["settlement"].to_list()):
                for i, r in df_tmp.iterrows():
                    pos_tmp = (r["latitude_dd"], r["longitude_dd"])
                    if row["area"] == r["settlement"]:
                        D_tmp = GD(pos_0, pos_tmp).km
                        if D_tmp < D_km:
                            region = r["region"]
                            municipality = r["municipality"]
                            settlement = r["settlement"]
                            settelment_type = r["type"]
                            population = r["population"]
                            children = r["children"]
                            D_km = D_tmp
            else:
                for i, r in df_tmp.iterrows():
                    pos_tmp = (r["latitude_dd"], r["longitude_dd"])
                    D_tmp = GD(pos_0, pos_tmp).km
                    if D_tmp < D_km:
                            region = r["region"]
                            municipality = r["municipality"]
                            settlement = r["settlement"]
                            settelment_type = r["type"]
                            population = r["population"]
                            children = r["children"]
                            D_km = D_tmp
            df_adr_s.at[index, "region_found"] = region
            df_adr_s.at[index, "municipality_found"] = municipality
            df_adr_s.at[index, "settlement_found"] = settlement
            df_adr_s.at[index, "settelment_type_found"] = settelment_type
            df_adr_s.at[index, "population_found"] = population
            df_adr_s.at[index, "children_found"] = children
            df_adr_s.at[index, "D_km_found"] = round(D_km, 3)
            n += 1
            clear_output()
            print(f"обработана строка {n} из {df_adr_s.shape[0]} при лимите {lim}\t\t{round(n*100/df_adr_s.shape[0], 3)} %", flush=True)
        else:
            break
    except Exception as e:
        print(f"строка {n} ошибка {e}")
print("сохранение")
df_adr_s.to_csv("C:/Users/Admin/Desktop/RESULT_2.csv", sep=";", encoding="utf-8-sig", index=False)
print("результат сохранён")

обработана строка 500 из 963782 при лимите 500		0.052 %
результат сохранён
CPU times: total: 7.62 s
Wall time: 38.5 s


In [57]:
%%time

# совсем уже аццкий код-3, поиск с учётом названия населённого пункта, а если такого нет - только по координатам, статус поиска и совпадение букв

# необходимое только для кода-3 поле
df_adr_s["sf/e"] = ""


n = 0
lim = 5000
for index, row in df_adr_s.iterrows():
    sf = ""
    try:
        if index < lim:
            D_km = 100500
            region = "no_data"
            municipality = "no_data"
            settlement = "no_data"
            settelment_type = "no_data"
            population = "no_data"
            children = "no_data"
            pos_0 = (row["lat"], row["lon"])
            df_tmp = df_big_pop[df_big_pop["region"] == row["reg_rosstat"]]
            if row["area"] in (df_tmp["settlement"].to_list()):
                sf = "1"
                k = 0
                for i, r in df_tmp.iterrows():
                    pos_tmp = (r["latitude_dd"], r["longitude_dd"])
                    if row["area"] == r["settlement"]:
                        k += 1
                        D_tmp = GD(pos_0, pos_tmp).km
                        if D_tmp < D_km:
                            region = r["region"]
                            municipality = r["municipality"]
                            settlement = r["settlement"]
                            settelment_type = r["type"]
                            population = r["population"]
                            children = r["children"]
                            D_km = D_tmp
                sf = sf + "_" + str(k)
            else:
                sf = "0"
                l = 0
                for i, r in df_tmp.iterrows():
                    pos_tmp = (r["latitude_dd"], r["longitude_dd"])
                    D_tmp = GD(pos_0, pos_tmp).km
                    l += 1
                    if D_tmp < D_km:
                        region = r["region"]
                        municipality = r["municipality"]
                        settlement = r["settlement"]
                        settelment_type = r["type"]
                        population = r["population"]
                        children = r["children"]
                        D_km = D_tmp
                # срвнение названий населённых пунктов
                row_list = list(row["area"])
                r_list = list(settlement)
                lst1 = row_list
                lst2 = r_list
                if len(row_list) < len(r_list):
                    lst1 = r_list
                    lst2 = row_list
                w_par = round(len([x for x in lst1 if x in set(lst2)])/len(lst1), 2)
                sf = sf + "_" + str(l) + "_" + str(w_par)
            df_adr_s.at[index, "region_found"] = region
            df_adr_s.at[index, "municipality_found"] = municipality
            df_adr_s.at[index, "settlement_found"] = settlement
            df_adr_s.at[index, "settelment_type_found"] = settelment_type
            df_adr_s.at[index, "population_found"] = population
            df_adr_s.at[index, "children_found"] = children
            df_adr_s.at[index, "D_km_found"] = round(D_km, 3)
            df_adr_s.at[index, "sf/e"] = sf
            n += 1
            clear_output()
            print(f"обработана строка {n} из {df_adr_s.shape[0]} при лимите {lim}\t{round(n*100/df_adr_s.shape[0], 3)} %\t({round(n*100/lim, 3)} %)", flush=True)
        else:
            break
    except Exception as e:
        print(f"строка {n} ошибка {e}")
        df_adr_s.at[index, "sf/e"] = e
print("сохранение")
df_adr_s.to_csv("C:/Users/Admin/Desktop/RESULT_3.csv", sep=";", encoding="utf-8-sig", index=False)
print("результат сохранён")

обработана строка 5000 из 963782 при лимите 5000	0.519 %	(100.0 %)
сохранение
результат сохранён
CPU times: total: 1min 4s
Wall time: 4min 58s


In [58]:
# df_adr_s.loc[df_adr_s["reg_rosstat"] == "москва", "population"] = 13104177
df_adr_s

,mo_oid,depart_oid,room_oid,lat,lon,reg,pref,area,reg_rosstat,region_found,municipality_found,settlement_found,settelment_type_found,population_found,children_found,D_km_found,sf/e
0,1.2.643.5.1.13.13.12.2.42.3982,1.2.643.5.1.13.13.12.2.42.3982.0.87331,1.2.643.5.1.13.13.12.2.42.3982.0.87331.26572,53.751255,87.136178,кемеровскаяобласть,г,новокузнецк,кемеровскаяобласть,кемеровскаяобласть,новокузнецкий,новокузнецк,г,551919,122291,0.695,1_1
1,1.2.643.5.1.13.13.12.2.42.3982,1.2.643.5.1.13.13.12.2.42.3982.0.87331,1.2.643.5.1.13.13.12.2.42.3982.0.87331.26573,53.751255,87.136178,кемеровскаяобласть,г,новокузнецк,кемеровскаяобласть,кемеровскаяобласть,новокузнецкий,новокузнецк,г,551919,122291,0.695,1_1
2,1.2.643.5.1.13.13.12.2.42.3982,1.2.643.5.1.13.13.12.2.42.3982.0.87331,1.2.643.5.1.13.13.12.2.42.3982.0.87331.26421,53.751255,87.136178,кемеровскаяобласть,г,новокузнецк,кемеровскаяобласть,кемеровскаяобласть,новокузнецкий,новокузнецк,г,551919,122291,0.695,1_1
3,1.2.643.5.1.13.13.12.2.42.3982,1.2.643.5.1.13.13.12.2.42.3982.0.87331,1.2.643.5.1.13.13.12.2.42.3982.0.87331.33475,53.751255,87.136178,кемеровскаяобласть,г,новокузнецк,кемеровскаяобласть,кемеровскаяобласть,новокузнецкий,новокузнецк,г,551919,122291,0.695,1_1
4,1.2.643.5.1.13.13.12.2.42.3982,1.2.643.5.1.13.13.12.2.42.3982.0.87331,1.2.643.5.1.13.13.12.2.42.3982.0.87331.33476,53.751255,87.136178,кемеровскаяобласть,г,новокузнецк,кемеровскаяобласть,кемеровскаяобласть,новокузнецкий,новокузнецк,г,551919,122291,0.695,1_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
963777,no_data,1.2.643.5.1.13.13.12.2.52.5060.0.454246,no_data,57.129626,45.145670,нижегородскаяобласть,рп,красныебаки,нижегородскаяобласть,,,,,,,,
963778,no_data,1.2.643.5.1.13.13.12.2.77.12482.0.461797,no_data,0.000000,0.000000,no_data,no_data,no_data,no_data,,,,,,,,
963779,no_data,1.2.643.5.1.13.13.12.2.77.1270.0.471279,no_data,0.000000,0.000000,no_data,no_data,no_data,no_data,,,,,,,,
963780,no_data,1.2.643.5.1.13.13.12.2.71.15515.0.450889,no_data,53.978838,38.530360,тульскаяобласть,г,кимовск,тульскаяобласть,,,,,,,,


In [28]:
df_adr_s[(df_adr_s["area"] == "мирный") & (df_adr_s["reg_rosstat"] == "саратовскаяобласть")]

,mo_oid,depart_oid,room_oid,lat,lon,reg,pref,area,reg_rosstat,region_found,municipality_found,settlement_found,settelment_type_found,population_found,children_found,D_km_found,sf/e
1941,1.2.643.5.1.13.13.12.2.64.6675,1.2.643.5.1.13.13.12.2.64.6675.0.175644,1.2.643.5.1.13.13.12.2.64.6675.0.175644.644904,52.219827,45.417680,саратовскаяобласть,п,мирный,саратовскаяобласть,саратовскаяобласть,петровскийрайон,мирный,п,321,73,0.477,1_4
237819,1.2.643.5.1.13.13.12.2.64.6563,1.2.643.5.1.13.13.12.2.64.6563.0.177662,1.2.643.5.1.13.13.12.2.64.6563.0.177662.639425,52.215617,48.889965,саратовскаяобласть,п,мирный,саратовскаяобласть,,,,,,,,
286954,1.2.643.5.1.13.13.12.2.64.6670,1.2.643.5.1.13.13.12.2.64.6670.0.524835,1.2.643.5.1.13.13.12.2.64.6670.0.524835.665054,50.908032,48.524207,саратовскаяобласть,п,мирный,саратовскаяобласть,,,,,,,,
286993,1.2.643.5.1.13.13.12.2.64.6670,1.2.643.5.1.13.13.12.2.64.6670.0.222278,1.2.643.5.1.13.13.12.2.64.6670.0.222278.664159,50.901682,48.522572,саратовскаяобласть,п,мирный,саратовскаяобласть,,,,,,,,
290696,1.2.643.5.1.13.13.12.2.64.6619,1.2.643.5.1.13.13.12.2.64.6619.0.174467,1.2.643.5.1.13.13.12.2.64.6619.0.174467.666510,50.352335,48.118294,саратовскаяобласть,п,мирный,саратовскаяобласть,,,,,,,,
406940,1.2.643.5.1.13.13.12.2.64.6621,1.2.643.5.1.13.13.12.2.64.6621.0.176164,1.2.643.5.1.13.13.12.2.64.6621.0.176164.289520,51.366284,48.660117,саратовскаяобласть,п,мирный,саратовскаяобласть,,,,,,,,


In [91]:
str = "ОЧЕНЬ ДЛИННОЕ НАИМЕНОВАНИЕ НЕКОТОРОГО МЕДИЦИНСКОГО УЧРЕЖДЕНИЯ"
str_list = str.split(" ")
new_str = " ".join(str_list[:2])
for element in str.split(" ")[2:-2]:
    element_short = " " + "".join([x for x in element][:3]) + "-" + "".join([x for x in element][-1])
    new_str += element_short
new_str = new_str + " " + str_list[-2] + " " + str_list[-1]
new_str

'ОЧЕНЬ ДЛИННОЕ НАИ-Е НЕК-О МЕДИЦИНСКОГО УЧРЕЖДЕНИЯ'

In [16]:
df_address.head(6)

,id,mo_region_name,mo_oid,mo_name_short,sp_oid,sp_name,fp_oid,fp_name,fp_type_name,bld_uuid,bld_id,bld_name,bld_address
0,ID записи справочника,Субъект РФ,OID медицинской организации,Краткое наименование медицинской организации,OID структурного подразделения,Наименование структурного подразделения,OID отделения/кабинета,Наименование отделения/кабинета,Наименование типа отделения/кабинета,GUID здания,ID здания,Наименование здания,Адрес здания
1,1,Свердловская область,1.2.643.5.1.13.13.12.2.66.22754,"ООО ""МКС ""БЕЛАЯ ЛАДЬЯ""",1.2.643.5.1.13.13.12.2.66.22754.0.460317,Фитнес центр,1.2.643.5.1.13.13.12.2.66.22754.0.460317.454771,NaN,Кабинет лечебной физкультуры,029075ac-3ca5-4bf7-8ef8-aa7d08307d90,163361,Фитнес центр,Свердловская Область г Екатеринбург ул Комсомо...
2,2,Республика Башкортостан,1.2.643.5.1.13.13.12.2.2.118,ГБУЗ РБ ГКБ № 5 Г.УФА,1.2.643.5.1.13.13.12.2.2.118.0.20566,Поликлиническое отделение 1 (Цюрупы 84),1.2.643.5.1.13.13.12.2.2.118.0.20566.15922,NaN,Кабинеты врачей гематологов,60152184-63e2-4cfb-8722-9d4bbbe7aa81,10943,Здание поликлиники (Цюрупы 84),Республика Башкортостан г Уфа ул Цюрупы 84
3,3,Кемеровская область,1.2.643.5.1.13.13.12.2.42.3939,ГАУЗ КО КККДЦ,1.2.643.5.1.13.13.12.2.42.3939.0.130687,Поликлиника № 1 Терапевтическое отделение 1,1.2.643.5.1.13.13.12.2.42.3939.0.130687.189120,NaN,Терапевтические,5be94957-5695-4262-abaa-94ec5c1e9cda,59179,ГАУЗ ККДЦ Поликлиника 1,Кемеровская область г Кемерово пр-кт Ленинград...
4,4,Архангельская область,1.2.643.5.1.13.13.12.2.29.2693,"ГБУЗ ""АГКБ № 7""",1.2.643.5.1.13.13.12.2.29.2693.0.63940,Педиатрическое отделение,1.2.643.5.1.13.13.12.2.29.2693.0.63940.55102,NaN,Педиатра участкового кабинет,c854d0bc-dafc-48d5-9129-1f9fc4c9f403,2527,нежилое помещение на ул Приморская,Архангельская Область г Архангельск ул Приморс...
5,5,Вологодская область,1.2.643.5.1.13.13.12.2.35.3271,"БУЗ ВО ""ТАРНОГСКАЯ ЦРБ""",1.2.643.5.1.13.13.12.2.35.3271.0.62159,Поликлиника,1.2.643.5.1.13.13.12.2.35.3271.0.62159.56147,NaN,Зубопротезные,98d839c1-b9a5-4c37-abcf-61757aefb5d6,32211,Здание больницы,Вологодская с Тарногский Городок ул Погранична...


In [7]:
import requests
from io import StringIO
import pandas as pd
from bs4 import BeautifulSoup
import re
from lxml import html
import time

In [8]:
site = "https://geoadm.com"
reg_dict = {
    "/naselennye-punkty-moskovskoy-oblasti.html": "Московская область",
    "/naselennye-punkty-krasnodarskogo-kraya.html": "Краснодарский край",
    "/naselennye-punkty-sverdlovskoy-oblasti.html": "Свердловская область",
    "/naselennye-punkty-rostovskoy-oblasti.html": "Ростовская область",
    "/naselennye-punkty-respubliki-bashkortostan.html": "Республика Башкортостан",
    "/naselennye-punkty-respubliki-tatarstan.html": "Республика Татарстан",
    "/naselennye-punkty-chelyabinskoy-oblasti.html": "Челябинская область",
    "/naselennye-punkty-nizhegorodskoy-oblasti.html": "Нижегородская область",
    "/naselennye-punkty-samarskoy-oblasti.html": "Самарская область",
    "/naselennye-punkty-respubliki-dagestan.html": "Республика Дагестан",
    "/naselennye-punkty-krasnoyarskogo-kraya.html": "Красноярский край",
    "/naselennye-punkty-stavropolskogo-kraya.html": "Ставропольский край",
    "/naselennye-punkty-novosibirskoy-oblasti.html": "Новосибирская область",
    "/naselennye-punkty-kemerovskoy-oblasti.html": "Кемеровская область",
    "/naselennye-punkty-permskogo-kraya.html": "Пермский край",
    "/naselennye-punkty-volgogradskoy-oblasti.html": "Волгоградская область",
    "/naselennye-punkty-saratovskoy-oblasti.html": "Саратовская область",
    "/naselennye-punkty-irkutskoy-oblasti.html": "Иркутская область",
    "/naselennye-punkty-voronezhskoy-oblasti.html": "Воронежская область",
    "/naselennye-punkty-altaiskogo-kraya.html": "Алтайский край",
    "/naselennye-punkty-orenburgskoy-oblasti.html": "Оренбургская область",
    "/naselennye-punkty-omskoy-oblasti.html": "Омская область",
    "/naselennye-punkty-respubliki-krym.html": "Республика Крым",
    "/naselennye-punkty-leningradskoy-oblasti.html": "Ленинградская область",
    "/naselennye-punkty-primorskogo-kraya.html": "Приморский край",
    "/naselennye-punkty-hanty-mansiyskogo-avtonomnogo-okruga.html": "Ханты-Мансийский автономный округ - Югра",
    "/naselennye-punkty-tyumenskoy-oblasti.html": "Тюменская область",
    "/naselennye-punkty-belgorodskoy-oblasti.html": "Белгородская область",
    "/naselennye-punkty-chechenskoy-respubliki.html": "Чеченская Республика",
    "/naselennye-punkty-udmurtskoy-respubliki.html": "Удмуртская Республика",
    "/naselennye-punkty-tulskoy-oblasti.html": "Тульская область",
    "/naselennye-punkty-vladimirskoy-oblasti.html": "Владимирская область",
    "/naselennye-punkty-habarovskogo-kraya.html": "Хабаровский край",
    "/naselennye-punkty-penzenskoy-oblasti.html": "Пензенская область",
    "/naselennye-punkty-kirovskoy-oblasti.html": "Кировская область",
    "/naselennye-punkty-tverskoy-oblasti.html": "Тверская область",
    "/naselennye-punkty-yaroslavskoy-oblasti.html": "Ярославская область",
    "/naselennye-punkty-ulianovskoy-oblasti.html": "Ульяновская область",
    "/naselennye-punkty-chuvashskoy-respubliki.html": "Чувашская Республика",
    "/naselennye-punkty-bryanskoy-oblasti.html": "Брянская область",
    "/naselennye-punkty-vologodskoy-oblasti.html": "Вологодская область",
    "/naselennye-punkty-lipetskoy-oblasti.html": "Липецкая область",
    "/naselennye-punkty-ryazanskoy-oblasti.html": "Рязанская область",
    "/naselennye-punkty-kurskoy-oblasti.html": "Курская область",
    "/naselennye-punkty-arhangelskoy-oblasti.html": "Архангельская область",
    "/naselennye-punkty-tomskoy-oblasti.html": "Томская область",
    "/naselennye-punkty-zabaikalskogo-kraya.html": "Забайкальский край",
    "/naselennye-punkty-kaliningradskoy-oblasti.html": "Калининградская область",
    "/naselennye-punkty-kaluzhskoy-oblasti.html": "Калужская область",
    "/naselennye-punkty-astrahanskoy-oblasti.html": "Астраханская область",
    "/naselennye-punkty-tambovskoy-oblasti.html": "Тамбовская область",
    "/naselennye-punkty-ivanovskoy-oblasti.html": "Ивановская область",
    "/naselennye-punkty-respubliki-buryatiya.html": "Республика Бурятия",
    "/naselennye-punkty-yakutii.html": "Республика Саха (Якутия)",
    "/naselennye-punkty-smolenskoy-oblasti.html": "Смоленская область",
    "/naselennye-punkty-kabardino-balkarskoy-respubliki.html": "Кабардино-Балкарская Республика",
    "/naselennye-punkty-kurganskoy-oblasti.html": "Курганская область",
    "/naselennye-punkty-respubliki-komi.html": "Республика Коми",
    "/naselennye-punkty-amurskoy-oblasti.html": "Амурская область",
    "/naselennye-punkty-respubliki-mordoviya.html": "Республика Мордовия",
    "/naselennye-punkty-murmanskoy-oblasti.html": "Мурманская область",
    "/naselennye-punkty-orlovskoy-oblasti.html": "Орловская область",
    "/naselennye-punkty-respubliki-severnaya-osetiya-alaniya.html": "Республика Северная Осетия - Алания",
    "/naselennye-punkty-respubliki-mariy-el.html": "Республика Марий Эл",
    "/naselennye-punkty-kostromskoy-oblasti.html": "Костромская область",
    "/naselennye-punkty-pskovskoy-oblasti.html": "Псковская область",
    "/naselennye-punkty-respubliki-kareliya.html": "Республика Карелия",
    "/naselennye-punkty-novgorodskoy-oblasti.html": "Новгородская область",
    "/naselennye-punkty-yamalo-nenetskogo-avtonomnogo-okruga.html": "Ямало-Ненецкий автономный округ",
    "/naselennye-punkty-respubliki-hakasiya.html": "Республика Хакасия",
    "/naselennye-punkty-respubliki-ingushetiya.html": "Республика Ингушетия",
    "/naselennye-punkty-sahalinskoy-oblasti.html": "Сахалинская область",
    "/naselennye-punkty-karachaevo-cherkesskoy-respubliki.html": "Карачаево-Черкесская Республика",
    "/naselennye-punkty-respubliki-adygeya.html": "Республика Адыгея",
    "/naselennye-punkty-respubliki-tyva.html": "Республика Тыва",
    "/naselennye-punkty-kamchatskogo-kraya.html": "Камчатский край",
    "/naselennye-punkty-respubliki-kalmykiya.html": "Республика Калмыкия",
    "/naselennye-punkty-respubliki-altay.html": "Республика Алтай",
    "/naselennye-punkty-evreiskoy-avtonomnoy-oblasti.html": "Еврейская автономная область",
    "/naselennye-punkty-magadanskoy-oblasti.html": "Магаданская область",
    "/naselennye-punkty-chukotskogo-avtonomniy-okruga.html": "Чукотский автономный округ",
    "/naselennye-punkty-nenetskogo-avtonomniy-okruga.html": "Ненецкий автономный округ",
}

In [9]:
%%time

df_new_population = pd.DataFrame()
for key, value in reg_dict.items():
    try:
        df_tmp = pd.read_html(site + key, index_col=None, header=0, encoding="utf-8")[1].drop("Unnamed: 0", axis=1)
        df_tmp.insert(0, "region", value)
        df_tmp.rename(columns={"Название": "area", "Тип": "prefix", "Население": "population", "Округ/Район": "municipality", "Округ/район": "municipality"},inplace=True)
        df_new_population = pd.concat([df_new_population, df_tmp])
        print(f"сделано {value}")
    except Exception as e:
        print(f"{key} трабл: {e}")
    # break
df_new_population = df_new_population.reset_index(drop=True)
df_new_population["population"] = df_new_population["population"].fillna(0)
df_new_population["population"] = df_new_population["population"].replace(r"\s+", "", regex=True)
df_new_population["population"] = df_new_population["population"].astype("int")
df_new_population

сделано Московская область
сделано Краснодарский край
сделано Свердловская область
сделано Ростовская область
сделано Республика Башкортостан
сделано Республика Татарстан
сделано Челябинская область
сделано Нижегородская область
сделано Самарская область
сделано Республика Дагестан
сделано Красноярский край
сделано Ставропольский край
сделано Новосибирская область
сделано Кемеровская область
сделано Пермский край
сделано Волгоградская область
сделано Саратовская область
сделано Иркутская область
сделано Воронежская область
сделано Алтайский край
сделано Оренбургская область
сделано Омская область
сделано Республика Крым
сделано Ленинградская область
сделано Приморский край
сделано Ханты-Мансийский автономный округ - Югра
сделано Тюменская область
сделано Белгородская область
сделано Чеченская Республика
сделано Удмуртская Республика
сделано Тульская область
сделано Владимирская область
сделано Хабаровский край
сделано Пензенская область
сделано Кировская область
сделано Тверская област

,region,area,prefix,population,municipality
0,Московская область,Балашиха,город,507307,Городской округ Балашиха
1,Московская область,Подольск,город,309250,Городской округ Подольск
2,Московская область,Химки,город,257757,Городской округ Химки
3,Московская область,Мытищи,город,245643,Городской округ Мытищи
4,Московская область,Королёв,город,225299,Городской округ Королёв
...,...,...,...,...,...
155266,Магаданская область,Беличан,пгт (рабочий посёлок),0,Сусуманский городской округ
155267,Магаданская область,Обо,посёлок,0,Тенькинский городской округ
155268,Магаданская область,Верхний Ат-Урях,пгт (рабочий посёлок),0,Ягоднинский городской округ
155269,Магаданская область,имени Горького,посёлок,0,Ягоднинский городской округ


In [10]:
df_new_pop = df_new_population.copy()
df_new_pop["_region"] = df_new_pop["region"].str.lower().str.replace(r'\W', '', regex=True)
df_new_pop["_area"] = df_new_pop["area"].str.lower().str.replace(r'\W', '', regex=True).str.replace('ё', 'е')
df_new_pop["_region"] = df_new_pop["_region"].str.replace('республикакрымa', 'республикакрым')
# df_new_pop.head(3)
df_new_pop_test = df_new_pop[["_region", "_area", "prefix"]]
df_new_pop_test

,_region,_area,prefix
0,московскаяобласть,балашиха,город
1,московскаяобласть,подольск,город
2,московскаяобласть,химки,город
3,московскаяобласть,мытищи,город
4,московскаяобласть,королев,город
...,...,...,...
155266,магаданскаяобласть,беличан,пгт (рабочий посёлок)
155267,магаданскаяобласть,обо,посёлок
155268,магаданскаяобласть,верхнийатурях,пгт (рабочий посёлок)
155269,магаданскаяобласть,именигорького,посёлок


In [11]:
df_big_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155921 entries, 0 to 155920
Data columns (total 13 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   region         155921 non-null  object 
 1   municipality   155921 non-null  object 
 2   settlement     155921 non-null  object 
 3   type           155921 non-null  object 
 4   population     155921 non-null  int64  
 5   children       155921 non-null  int64  
 6   latitude_dms   155921 non-null  object 
 7   longitude_dms  155921 non-null  object 
 8   latitude_dd    155921 non-null  float64
 9   longitude_dd   155921 non-null  float64
 10  oktmo          155920 non-null  float64
 11  dadata         155921 non-null  int64  
 12  rosstat        155921 non-null  int64  
dtypes: float64(3), int64(4), object(6)
memory usage: 15.5+ MB


In [12]:
# df_big_pop.head(3)
df_big_pop_test = df_big_pop[["region", "settlement", "type"]]
df_big_pop_test["settlement"] = df_big_pop_test["settlement"].str.replace('ё', 'е')
df_big_pop_test["region"] = df_big_pop_test["region"].str.replace('республикакрыма', 'республикакрым')
df_big_pop_test

,region,settlement,type
0,орловскаяобласть,колонтаева,д
1,республикакрым,пушкино,с
2,липецкаяобласть,барятино,с
3,тверскаяобласть,хилово,д
4,томскаяобласть,басмасово,д
...,...,...,...
155916,ульяновскаяобласть,ребровка,д
155917,псковскаяобласть,шадрица,д
155918,псковскаяобласть,овчинниково,д
155919,тульскаяобласть,новогуровский,рп


In [17]:
df_test_0 = df_big_pop_test.merge(df_new_pop_test, left_on=["region", "settlement"], right_on=["_region", "_area"], how="outer")
df_test_na = df_test_0[df_test_0["region"].isna() | df_test_0["settlement"].isna() | df_test_0["_region"].isna() | df_test_0["_area"].isna()].reset_index(drop=True).drop_duplicates()
df_test_na

,region,settlement,type,_region,_area,prefix
0,псковскаяобласть,хибины,д,NaN,NaN,NaN
1,свердловскаяобласть,новыекривки,д,NaN,NaN,NaN
2,вологодскаяобласть,подлесное,д,NaN,NaN,NaN
3,республикадагестан,гавутли,с,NaN,NaN,NaN
4,свердловскаяобласть,унжепавинск,д,NaN,NaN,NaN
...,...,...,...,...,...,...
7692,NaN,NaN,NaN,магаданскаяобласть,беличан,пгт (рабочий посёлок)
7693,NaN,NaN,NaN,магаданскаяобласть,обо,посёлок
7694,NaN,NaN,NaN,магаданскаяобласть,верхнийатурях,пгт (рабочий посёлок)
7695,NaN,NaN,NaN,магаданскаяобласть,именигорького,посёлок


In [28]:
# численность населения
df_new_population
df_new_pop_group = pd.DataFrame(df_new_population.groupby("region")["population"].sum()).reset_index()
df_new_pop_group["region"] = df_new_pop_group["region"].str.lower().str.replace(r'\W', '', regex=True).str.replace('ё', 'е')
df_new_pop_group = df_new_pop_group.rename(columns = {"population": "pop_data_2"})
df_big_pop_group = pd.DataFrame(df_big_pop.groupby("region")["population"].sum()).reset_index()
df_big_pop_group = df_big_pop_group.rename(columns = {"population": "pop_data_1"})
df_new_pop_group

,region,pop_data_2
0,алтайскийкрай,2404826
1,амурскаяобласть,809257
2,архангельскаяобласть,1147490
3,астраханскаяобласть,999322
4,белгородскаяобласть,1564443
...,...,...
75,челябинскаяобласть,3492429
76,чеченскаяреспублика,1368233
77,чувашскаяреспублика,1284684
78,ямалоненецкийавтономныйокруг,535816


In [52]:
df_pop_group = df_big_pop_group.merge(df_new_pop_group, on="region", how="outer")
df_pop_group[["pop_data_1", "pop_data_2"]] = df_pop_group[["pop_data_1", "pop_data_2"]].fillna(0)
df_pop_group[["pop_data_1", "pop_data_2"]] = df_pop_group[["pop_data_1", "pop_data_2"]].astype("int")
df_pop_group["d_2/d_1(%)"] = round(df_pop_group["pop_data_2"]*100/df_pop_group["pop_data_1"], 1)
df_pop_group["d_2/d_1(%)"] = df_pop_group["d_2/d_1(%)"].fillna(0)

In [53]:
df_pop_group

,region,pop_data_1,pop_data_2,d_2/d_1(%)
0,алтайскийкрай,2332813,2404826,103.1
1,амурскаяобласть,786213,809257,102.9
2,архангельскаяобласть,1100247,1147490,104.3
3,астраханскаяобласть,1016036,999322,98.4
4,байконур,0,0,0.0
...,...,...,...,...
81,чеченскаяреспублика,1456949,1368233,93.9
82,чувашскаяреспублика,1221431,1284684,105.2
83,чукотскийавтономныйокруг,49790,0,0.0
84,ямалоненецкийавтономныйокруг,545045,535816,98.3


In [59]:
df_new_mean = pd.DataFrame(df_new_pop.groupby("prefix")["population"].mean()).reset_index()
df_big_mean = pd.DataFrame(df_big_pop.groupby("type")["population"].mean()).reset_index()

In [60]:
df_new_mean

,prefix,population
0,Город,28267.214286
1,ПГТ,3327.500000
2,ПГТ (дачный посёлок),6380.222222
3,ПГТ (курортный посёлок),2105.000000
4,аал,259.979167
...,...,...
89,станция,201.799270
90,улус,423.743169
91,усадьба,68.000000
92,участок,78.258621


In [61]:
df_big_mean

,type,population
0,c,48.250000
1,x,26.000000
2,аал,208.769231
3,автодорога,0.000000
4,арбан,116.000000
5,аул,1129.700508
6,высел,33.169811
7,г,86902.946087
8,городок,1875.714286
9,гп,3763.833333


In [62]:
df_big_pop[df_big_pop["type"] == "рн"]

,region,municipality,settlement,type,population,children,latitude_dms,longitude_dms,latitude_dd,longitude_dd,oktmo,dadata,rosstat
2103,астраханскаяобласть,городастрахань,ленинский,рн,148717,28855,46.23.40,048.06.24,46.394444,48.106667,1.270100e+07,1,0
6689,астраханскаяобласть,городастрахань,трусовский,рн,115208,23282,46.21.06,047.59.44,46.351667,47.995556,1.270100e+07,1,0
21811,санктпетербург,центральный,центральный,рн,0,0,59.55.36,030.21.31,59.926667,30.358611,4.034300e+07,1,0
26939,республикабашкортостан,уфа,октябрьский,рн,240708,44727,54.45.57,056.01.53,54.765833,56.031389,8.070100e+07,1,0
37700,красноярскийкрай,норильск,талнах,рн,47307,11204,69.29.21,088.24.24,69.489167,88.406667,4.729000e+06,1,0
37837,республикабашкортостан,уфа,советский,рн,177865,29691,54.44.13,055.59.10,54.736944,55.986111,8.070100e+07,1,0
41100,республикадагестан,махачкала,кировскийрайон,рн,0,0,42.59.30,047.25.04,42.991667,47.417778,8.270100e+10,1,0
45289,республикабашкортостан,уфа,кировский,рн,156820,31455,54.41.12,055.58.51,54.686667,55.980833,8.070100e+07,1,0
48669,республикабашкортостан,уфа,демский,рн,70365,14413,54.41.56,055.55.01,54.698889,55.916944,8.070100e+07,1,0
58184,республикабашкортостан,уфа,калининский,рн,199883,38010,54.45.51,056.09.39,54.764167,56.160833,8.070100e+07,1,0
